<a href="https://colab.research.google.com/github/DeSlay/Growth_business_ecommerce/blob/main/02_Prediction_futur_achat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing
import os # manipulate paths
from datetime import date # supplies classes for manipulating dates
from time import strftime # convert time to a string as specified by the format argument

import seaborn as sns # to create visualisations
import matplotlib.pyplot as plt
%matplotlib inline
import plotly
import plotly.graph_objs as go
import cufflinks as cf

import io
from google.colab import files

In [6]:
uploaded = files.upload()

df = pd.read_csv(io.BytesIO(uploaded['output (2).csv']), sep = ',')

Saving output (2).csv to output (2).csv


In [13]:
df

,Unnamed: 0,id_order,invoice_number,id_customer,total_products,invoice_date,lastname,firstname,postcode,id_country,...,FirstOrder,UserType,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,Segment
0,0,7,2,6,22.95,2020-01-12 14:49:30,Azor,Gilliane,69007,France,...,202001,New,646,2,2,1,39.85,0,3,Mid-Value
1,1,372,364,6,16.90,2020-11-16 12:47:22,Azor,Gilliane,69007,France,...,202001,Existing,646,2,2,1,39.85,0,3,Mid-Value
2,2,444,436,889,32.80,2020-12-21 13:42:33,MEG,LEFEBVRE,59470,France,...,202012,New,567,2,2,1,49.70,0,3,Mid-Value
3,3,615,606,889,16.90,2021-02-03 14:42:15,MEG,LEFEBVRE,59470,France,...,202012,Existing,567,2,2,1,49.70,0,3,Mid-Value
4,4,480,472,950,14.90,2021-01-08 10:49:49,MARULAZ,Lauriane,92220,France,...,202101,New,545,2,2,1,46.70,0,3,Mid-Value
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2286,2286,1501,1493,1393,34.80,2021-10-19 06:13:16,Blanchard,Maïna,64250,France,...,202102,Existing,60,0,8,3,336.30,3,6,ULTRA-High-Value
2287,2287,1591,1583,1393,17.90,2021-11-19 07:56:41,Blanchard,Maïna,64250,France,...,202102,Existing,60,0,8,3,336.30,3,6,ULTRA-High-Value
2288,2288,2032,2024,1393,48.40,2022-03-20 06:31:49,Blanchard,Maïna,64250,France,...,202102,Existing,60,0,8,3,336.30,3,6,ULTRA-High-Value
2289,2289,2146,2138,1393,59.60,2022-05-14 08:02:15,Blanchard,Maïna,64250,France,...,202102,Existing,60,0,8,3,336.30,3,6,ULTRA-High-Value


In [ ]:
df['invoice_date'] = pd.to_datetime(df['invoice_date'])

df_customer_with_model = df.query("Frequency > 2 ").reset_index(drop=True)

df_9m = df_customer_with_model[(df_customer_with_model.invoice_date < pd.to_datetime('2021-12-01')) & (df_customer_with_model.invoice_date >= pd.to_datetime('2021-01-01'))].reset_index(drop=True)
df_next_purchase = df_customer_with_model[(df_customer_with_model.invoice_date >= pd.to_datetime('2021-12-01')) & (df_customer_with_model.invoice_date < pd.to_datetime('2022-08-01'))].reset_index(drop=True)

df_next_purchase

In [34]:

df_next_first_purchase = df_next_purchase.groupby('id_customer').invoice_date.min().reset_index()
df_next_first_purchase.columns = ['id_customer','FirstPurchaseDate']


# create a dataframe with customer id and last purchase date in tx_6m
df_last_purchase = df_9m.groupby('id_customer').invoice_date.max().reset_index()
df_last_purchase.columns = ['id_customer','LastPurchaseDate']

# merge two dataframes
df_purchase_dates = pd.merge(df_last_purchase,df_next_first_purchase,on='id_customer',how='left')

# calculate the time difference in days
df_purchase_dates['NextPurchaseDay'] = (df_purchase_dates['FirstPurchaseDate'] - df_purchase_dates['LastPurchaseDate']).dt.days

# merge with df_customers 
df_customer_with_model = pd.merge(df_customer_with_model, df_purchase_dates[['id_customer','NextPurchaseDay']],on='id_customer',how='left')

df_customer_with_model = df_customer_with_model[df_customer_with_model['id_customer'].notna()]

df_customer_with_model



,Unnamed: 0,id_order,invoice_number,id_customer,total_products,invoice_date,lastname,firstname,postcode,id_country,...,UserType,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,Segment,NextPurchaseDay
0,857,37,31,59,49.70,2020-02-21 14:39:16,JEAN,Gladys,97214,Martinique,...,New,540,2,3,1,159.60,2,5,High-Value,NaN
1,858,137,129,59,48.40,2020-05-02 21:09:50,JEAN,Gladys,97214,Martinique,...,Existing,540,2,3,1,159.60,2,5,High-Value,NaN
2,859,757,749,59,61.50,2021-03-02 11:01:07,JEAN,Gladys,97214,Martinique,...,Existing,540,2,3,1,159.60,2,5,High-Value,NaN
3,862,90,83,222,61.50,2020-04-03 05:57:51,Patissier,Prisca,97480,La_Reunion,...,New,628,2,3,1,157.88,2,5,High-Value,NaN
4,863,250,242,222,65.88,2020-08-10 12:46:41,Patissier,Prisca,97480,La_Reunion,...,Existing,628,2,3,1,157.88,2,5,High-Value,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,2286,1501,1493,1393,34.80,2021-10-19 06:13:16,Blanchard,Maïna,64250,France,...,Existing,60,0,8,3,336.30,3,6,ULTRA-High-Value,120.0
337,2287,1591,1583,1393,17.90,2021-11-19 07:56:41,Blanchard,Maïna,64250,France,...,Existing,60,0,8,3,336.30,3,6,ULTRA-High-Value,120.0
338,2288,2032,2024,1393,48.40,2022-03-20 06:31:49,Blanchard,Maïna,64250,France,...,Existing,60,0,8,3,336.30,3,6,ULTRA-High-Value,120.0
339,2289,2146,2138,1393,59.60,2022-05-14 08:02:15,Blanchard,Maïna,64250,France,...,Existing,60,0,8,3,336.30,3,6,ULTRA-High-Value,120.0


In [ ]:
df

In [83]:
#3. Data Wrangling
#Pour construire notre modèle, nous utilisons neuf mois de données comportementales pour prédire la date du premier achat des clients au cours des trois prochains mois. 
#Cela prendra également en compte les clients qui n'ont pas acheté. 
#Le but est de trouver le nombre de jours entre le dernier achat dans les données comportementales et le premier dans les prochaines données d'achat.

# create a dataset into two dataframes
df['invoice_date'] = pd.to_datetime(df['invoice_date'])

df_customer_with_model = df.copy()

df_9m = df_customer_with_model[(df_customer_with_model.invoice_date < pd.to_datetime('2021-12-01')) & (df_customer_with_model.invoice_date >= pd.to_datetime('2021-01-01'))].reset_index(drop=True)
df_next_purchase = df_customer_with_model[(df_customer_with_model.invoice_date >= pd.to_datetime('2021-12-01')) & (df_customer_with_model.invoice_date < pd.to_datetime('2022-08-01'))].reset_index(drop=True)

# create a dataframe at customers level for the prediction model 
'''
df_customers_prediction = pd.DataFrame(df_9m['id_customer'].unique())
df_customers_prediction.columns = ['id_customer']
#We now need to calculate our label, namely, the number of days between last purchase before the 1st of Septmeber 2011 (cut off date) and first purchase after that.
'''

# create a dataframe with customer id and first purchase date in df_next_purchase
df_next_first_purchase = df_next_purchase.groupby('id_customer').invoice_date.min().reset_index()
df_next_first_purchase.columns = ['id_customer','FirstPurchaseDate']

# create a dataframe with customer id and last purchase date in tx_6m
df_last_purchase = df_9m.groupby('id_customer').invoice_date.max().reset_index()
df_last_purchase.columns = ['id_customer','LastPurchaseDate']

# merge two dataframes
df_purchase_dates = pd.merge(df_last_purchase,df_next_first_purchase,on='id_customer',how='left')

# calculate the time difference in days
df_purchase_dates['NextPurchaseDay'] = (df_purchase_dates['FirstPurchaseDate'] - df_purchase_dates['LastPurchaseDate']).dt.days

# merge with df_customers 
df_customer_with_model = pd.merge(df_customer_with_model, df_purchase_dates[['id_customer','NextPurchaseDay']],on='id_customer',how='left')

# keep only the rows where CustomerID is not NaN
df_customer_with_model = df_customer_with_model[df_customer_with_model['id_customer'].notna()]

# replace NextPurchaseDay's NaN by 999
#df_customers = df_customers.fillna(999)

# print df_customers
df_customer_with_model.head()

'''
df_rfm = df[['id_customer','Recency', 'RecencyCluster', 'Frequency', 'FrequencyCluster', 'Revenue', 'RevenueCluster', 'OverallScore', 'Segment' ]]

#4.Feature Engineering

# merge df_rfm to df_customers
df_customers_diff = pd.merge(df_customer_with_model, df_rfm, on='id_customer')
'''

# create a dataframe with CustomerID and InvoiceDate
df_order = df_9m[['id_customer','invoice_date', 'lastname']]

# convert InvoiceDate from datetime to date
df_order['invoice_date'] = df_9m['invoice_date'].dt.date
df_order = df_order.sort_values(['id_customer','invoice_date'])

# drop duplicates
df_order = df_order.drop_duplicates(subset=['id_customer','invoice_date'],keep='first')

df_order['T1InvoiceDate'] = df_order.groupby('id_customer')['invoice_date'].shift(1)
df_order['T2InvoiceDate'] = df_order.groupby('id_customer')['invoice_date'].shift(2)
df_order['T3InvoiceDate'] = df_order.groupby('id_customer')['invoice_date'].shift(3)
df_order['T4InvoiceDate'] = df_order.groupby('id_customer')['invoice_date'].shift(4)

df_order.head(20)
# remove the duplicates
df_order = df_order.drop_duplicates(subset=['id_customer'],keep='last')

# remove NAs
df_order = df_order[df_order['id_customer'].notna()]

# print df_customers
df_order.head()

# calculate the difference in days for each invoice date
df_order['DiffOrder1'] = (df_order['invoice_date'] - df_order['T1InvoiceDate']).dt.days
df_order['DiffOrder2'] = (df_order['invoice_date'] - df_order['T2InvoiceDate']).dt.days
df_order['DiffOrder3'] = (df_order['invoice_date'] - df_order['T3InvoiceDate']).dt.days
df_order['DiffOrder4'] = (df_order['invoice_date'] - df_order['T4InvoiceDate']).dt.days

# print df_order
df_order


# calculate the mean number of days between orders for each customers
df_order['DayDiffMean'] = df_order.iloc[:,7:11].mean(axis=1)

# calculate the standard deviation of days between orders for each customers
df_order['DayDiffStd'] = df_order.iloc[:,7:11].std(axis=1)

# print df_order
df_order

# merge the results to the customer dataframe
df_customers_diff = pd.merge(df_customer_with_model, df_order[['id_customer', 'DiffOrder1','DiffOrder2','DiffOrder3','DiffOrder4','DayDiffMean','DayDiffStd']], on='id_customer')

#select just une seule ligne d'identifiant 
df_customers_diff.drop_duplicates(subset ="id_customer", keep = 'first', inplace=True) #supprimer doublons par rapport une valeur 

# print df_customers
df_customers_diff

,Unnamed: 0,id_order,invoice_number,id_customer,total_products,invoice_date,lastname,firstname,postcode,id_country,...,RevenueCluster,OverallScore,Segment,NextPurchaseDay,DiffOrder1,DiffOrder2,DiffOrder3,DiffOrder4,DayDiffMean,DayDiffStd
0,2,444,436,889,32.80,2020-12-21 13:42:33,MEG,LEFEBVRE,59470,France,...,0,3,Mid-Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,480,472,950,14.90,2021-01-08 10:49:49,MARULAZ,Lauriane,92220,France,...,0,3,Mid-Value,NaN,48.0,NaN,NaN,NaN,48.0,NaN
4,6,748,740,1425,16.90,2021-02-28 17:43:32,Chennak,Hind,33300,France,...,0,3,Mid-Value,NaN,77.0,NaN,NaN,NaN,77.0,NaN
6,8,855,847,1618,16.90,2021-03-22 17:02:21,ABDALLAH,Bibi Fatuma,35000,France,...,0,3,Mid-Value,NaN,54.0,NaN,NaN,NaN,54.0,NaN
8,10,858,850,1623,30.50,2021-03-23 13:04:36,chicoine,delphine,14800,France,...,0,3,Mid-Value,NaN,77.0,NaN,NaN,NaN,77.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,2255,889,881,1696,49.70,2021-03-29 13:08:24,Deho,Anna,6200,France,...,3,5,High-Value,139.0,104.0,160.0,NaN,NaN,132.0,39.597980
1316,2261,1330,1322,2370,77.50,2021-08-15 12:55:15,TAMBY,Sarasvati,97170,Guadeloupe,...,3,5,High-Value,139.0,55.0,NaN,NaN,NaN,55.0,NaN
1320,2265,82,76,224,61.50,2020-03-29 09:58:54,Segbaya,Kevin,95100,France,...,3,6,ULTRA-High-Value,14.0,104.0,226.0,318.0,NaN,216.0,107.349895
1329,2274,235,227,550,65.88,2020-07-24 03:07:26,Toussaint,Gilande,78500,France,...,3,6,ULTRA-High-Value,63.0,60.0,142.0,209.0,NaN,137.0,74.625733


In [85]:
df_customers_diff.to_csv('stats.csv', encoding = 'utf-8-sig') 
files.download('stats.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [55]:
#3. Data Wrangling
#Pour construire notre modèle, nous utilisons neuf mois de données comportementales pour prédire la date du premier achat des clients au cours des trois prochains mois. 
#Cela prendra également en compte les clients qui n'ont pas acheté. 
#Le but est de trouver le nombre de jours entre le dernier achat dans les données comportementales et le premier dans les prochaines données d'achat.

# create a dataset into two dataframes
df['invoice_date'] = pd.to_datetime(df['invoice_date'])

df_customer_with_model = df.query("Frequency > 2 ").reset_index(drop=True)

df_9m = df_customer_with_model[(df_customer_with_model.invoice_date < pd.to_datetime('2021-12-01')) & (df_customer_with_model.invoice_date >= pd.to_datetime('2021-01-01'))].reset_index(drop=True)
df_next_purchase = df_customer_with_model[(df_customer_with_model.invoice_date >= pd.to_datetime('2021-12-01')) & (df_customer_with_model.invoice_date < pd.to_datetime('2022-08-01'))].reset_index(drop=True)

# create a dataframe at customers level for the prediction model 
'''
df_customers_prediction = pd.DataFrame(df_9m['id_customer'].unique())
df_customers_prediction.columns = ['id_customer']
#We now need to calculate our label, namely, the number of days between last purchase before the 1st of Septmeber 2011 (cut off date) and first purchase after that.
'''

# create a dataframe with customer id and first purchase date in df_next_purchase
df_next_first_purchase = df_next_purchase.groupby('id_customer').invoice_date.min().reset_index()
df_next_first_purchase.columns = ['id_customer','FirstPurchaseDate']

# create a dataframe with customer id and last purchase date in tx_6m
df_last_purchase = df_9m.groupby('id_customer').invoice_date.max().reset_index()
df_last_purchase.columns = ['id_customer','LastPurchaseDate']

# merge two dataframes
df_purchase_dates = pd.merge(df_last_purchase,df_next_first_purchase,on='id_customer',how='left')

# calculate the time difference in days
df_purchase_dates['NextPurchaseDay'] = (df_purchase_dates['FirstPurchaseDate'] - df_purchase_dates['LastPurchaseDate']).dt.days

# merge with df_customers 
df_customer_with_model = pd.merge(df_customer_with_model, df_purchase_dates[['id_customer','NextPurchaseDay']],on='id_customer',how='left')

# keep only the rows where CustomerID is not NaN
df_customer_with_model = df_customer_with_model[df_customer_with_model['id_customer'].notna()]

# replace NextPurchaseDay's NaN by 999
#df_customers = df_customers.fillna(999)

# print df_customers
df_customer_with_model.head()

'''
df_rfm = df[['id_customer','Recency', 'RecencyCluster', 'Frequency', 'FrequencyCluster', 'Revenue', 'RevenueCluster', 'OverallScore', 'Segment' ]]

#4.Feature Engineering

# merge df_rfm to df_customers
df_customers_diff = pd.merge(df_customer_with_model, df_rfm, on='id_customer')
'''

# create a dataframe with CustomerID and InvoiceDate
df_order = df_9m[['id_customer','invoice_date', 'lastname']]

# convert InvoiceDate from datetime to date
df_order['invoice_date'] = df_9m['invoice_date'].dt.date
df_order = df_order.sort_values(['id_customer','invoice_date'])

# drop duplicates
df_order = df_order.drop_duplicates(subset=['id_customer','invoice_date'],keep='first')

df_order['T1InvoiceDate'] = df_order.groupby('id_customer')['invoice_date'].shift(1)
df_order['T2InvoiceDate'] = df_order.groupby('id_customer')['invoice_date'].shift(2)
df_order['T3InvoiceDate'] = df_order.groupby('id_customer')['invoice_date'].shift(3)
df_order['T4InvoiceDate'] = df_order.groupby('id_customer')['invoice_date'].shift(4)

df_order.head(20)
# remove the duplicates
df_order = df_order.drop_duplicates(subset=['id_customer'],keep='last')

# remove NAs
df_order = df_order[df_order['id_customer'].notna()]

# print df_customers
df_order.head()

# calculate the difference in days for each invoice date
df_order['DiffOrder1'] = (df_order['invoice_date'] - df_order['T1InvoiceDate']).dt.days
df_order['DiffOrder2'] = (df_order['invoice_date'] - df_order['T2InvoiceDate']).dt.days
df_order['DiffOrder3'] = (df_order['invoice_date'] - df_order['T3InvoiceDate']).dt.days
df_order['DiffOrder4'] = (df_order['invoice_date'] - df_order['T4InvoiceDate']).dt.days

# print df_order
df_order


# calculate the mean number of days between orders for each customers
df_order['DayDiffMean'] = df_order.iloc[:,7:11].mean(axis=1)

# calculate the standard deviation of days between orders for each customers
df_order['DayDiffStd'] = df_order.iloc[:,7:11].std(axis=1)

# print df_order
df_order

# merge the results to the customer dataframe
df_customers_diff = pd.merge(df_customer_with_model, df_order[['id_customer', 'DiffOrder1','DiffOrder2','DiffOrder3','DiffOrder4','DayDiffMean','DayDiffStd']], on='id_customer')

# print df_customers
df_customers_diff.head()

,Unnamed: 0,id_order,invoice_number,id_customer,total_products,invoice_date,lastname,firstname,postcode,id_country,...,RevenueCluster,OverallScore,Segment,NextPurchaseDay,DiffOrder1,DiffOrder2,DiffOrder3,DiffOrder4,DayDiffMean,DayDiffStd
0,857,37,31,59,49.7,2020-02-21 14:39:16,JEAN,Gladys,97214,Martinique,...,2,5,High-Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,858,137,129,59,48.4,2020-05-02 21:09:50,JEAN,Gladys,97214,Martinique,...,2,5,High-Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,859,757,749,59,61.5,2021-03-02 11:01:07,JEAN,Gladys,97214,Martinique,...,2,5,High-Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,865,100,93,257,34.8,2020-04-11 15:52:17,ANRETARD,Annabelle,97211,Martinique,...,2,5,High-Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,866,184,176,257,35.8,2020-06-08 13:18:07,ANRETARD,Annabelle,97211,Martinique,...,2,5,High-Value,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df_customers_diff

df_model['NextPurchaseDay'].describe()

#CREATE MODEL 

# remove the customers with only one purchase as we cannot consider them a frequent customers
df_customers = df_customers_diff[df_customers_diff['NextPurchaseDay'] != 999]

# print df_customers
df_customers.head(10)

# create a new data set as a copy of df_customers before applying get_dummies

df_model = df_customers[''].copy()

In [71]:
df_customers[column_for_model[13:26]].copy()

,product_quantity,InvoiceYearMonth,FirstOrder,UserType,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,Segment,NextPurchaseDay
0,1,202002,202002,New,540,2,3,1,159.6,2,5,High-Value,NaN
1,1,202005,202002,Existing,540,2,3,1,159.6,2,5,High-Value,NaN
2,1,202103,202002,Existing,540,2,3,1,159.6,2,5,High-Value,NaN
3,1,202004,202004,New,475,2,3,1,123.3,2,5,High-Value,NaN
4,2,202006,202004,Existing,475,2,3,1,123.3,2,5,High-Value,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,1,202110,202102,Existing,60,0,8,3,336.3,3,6,ULTRA-High-Value,120.0
274,1,202111,202102,Existing,60,0,8,3,336.3,3,6,ULTRA-High-Value,120.0
275,1,202203,202102,Existing,60,0,8,3,336.3,3,6,ULTRA-High-Value,120.0
276,1,202205,202102,Existing,60,0,8,3,336.3,3,6,ULTRA-High-Value,120.0


In [73]:
#CREATE MODEL 

# remove the customers with only one purchase as we cannot consider them a frequent customers
df_customers = df_customers_diff[df_customers_diff['NextPurchaseDay'] != 999]

# print df_customers
df_customers.head(10)

# create a new data set as a copy of df_customers before applying get_dummies
column_for_model = list(df_customers)

df_model = df_customers[column_for_model[13:26]].copy()

df_model = pd.get_dummies(df_model)

# create a feature for the next purchase range
df_model['NextPurchaseClass'] = 2
df_model.loc[df_model.NextPurchaseDay>78,'NextPurchaseClass'] = 1
df_model.loc[df_model.NextPurchaseDay>130,'NextPurchaseClass'] = 0

#df_model.drop_duplicates(subset ="id_customer", keep = 'first', inplace=True) #supprimer doublons par rapport une valeur 


# print df_model
df_model

#5.Selecting a Machine Learning Model
# Drop the NextPurchaseDay column


,product_quantity,InvoiceYearMonth,FirstOrder,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,NextPurchaseDay,UserType_Existing,UserType_New,Segment_High-Value,Segment_Mid-Value,Segment_ULTRA-High-Value,NextPurchaseClass
0,1,202002,202002,540,2,3,1,159.6,2,5,NaN,0,1,1,0,0,2
1,1,202005,202002,540,2,3,1,159.6,2,5,NaN,1,0,1,0,0,2
2,1,202103,202002,540,2,3,1,159.6,2,5,NaN,1,0,1,0,0,2
3,1,202004,202004,475,2,3,1,123.3,2,5,NaN,0,1,1,0,0,2
4,2,202006,202004,475,2,3,1,123.3,2,5,NaN,1,0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,1,202110,202102,60,0,8,3,336.3,3,6,120.0,1,0,0,0,1,1
274,1,202111,202102,60,0,8,3,336.3,3,6,120.0,1,0,0,0,1,1
275,1,202203,202102,60,0,8,3,336.3,3,6,120.0,1,0,0,0,1,1
276,1,202205,202102,60,0,8,3,336.3,3,6,120.0,1,0,0,0,1,1


In [78]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import GridSearchCV


#df_model = df_model.drop('NextPurchaseDay',axis=1)

# Select the prediction target
y = df_model.NextPurchaseClass

# Features selection
X = df_model.drop('NextPurchaseClass',axis=1)

# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 101)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

# create an array of models
models = []
models.append(("NB",GaussianNB()))
models.append(("RF",RandomForestClassifier()))
models.append(("Tree",DecisionTreeClassifier()))
models.append(("XGB",xgb.XGBClassifier()))

# measure the accuracy 
model_name = ['NB', 'RF', 'Tree', 'XGB'] 
results = []
for name,model in models:
    kfold = KFold(n_splits=2)
    cv_result = cross_val_score(model,X_train,y_train, cv = kfold, scoring = "accuracy")
    
    results.append(cv_result)
    columns = ['score_n1', 'score_n2']
    results_df =  pd.DataFrame(results, columns = columns)
    
# add the model column
results_df['model'] = model_name

# calculate the deviation score
results_df['deviation'] = results_df['score_n1']-results_df['score_n2']

# print
results_df

xgb_model = xgb.XGBClassifier(random_state = 101).fit(X_train, y_train)

print('Accuracy of XGBoost classifier on test set: {:.2f}'
       .format(xgb_model.score(X_test[X_train.columns], y_test)))

param_test1 = {'max_depth':range(0,20,2),
               'min_child_weight':range(0,10,2)}

gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier(random_state = 101), 
param_grid = param_test1, scoring='accuracy', n_jobs=-1, cv=4)

gsearch1.fit(X_train,y_train)
gsearch1.best_params_, gsearch1.best_score_

(208, 16) (208,)
(70, 16) (70,)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning:


2 fits failed out of a total of 2.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py", line 246, in fit
    X, y, np.unique(y), _refit=True, sample_weight=sample_weight
  File "/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py", line 402, in _partial_fit
    X, y = self._validate_data(X, y, reset=first_call)
  File "/usr/

Accuracy of XGBoost classifier on test set: 1.00


({'max_depth': 2, 'min_child_weight': 0}, 1.0)